# PETCA - Projeto de Análise de Contas de Energia com Aprendizado de Máquina e Redes Neurais

## Índice
- [Modelos Utilizados](#modelos-utilizados)
- [Importando Pacotes e Bibliotecas](#importando-os-pacotes-e-bibliotecas)
- [Importando os Datasets](#importando-os-datasets)
- [Análise Inicial dos Datasets](#analise-inicial-dos-datasets)
- [Análise Exploratória dos Dados](#aed)
- [Criando os Modelos](#criando-os-modelos)
- [Treinando os Modelos](#treinando-os-modelos)
- [Resultados os Modelos](#resultados-dos-modelos)
    - [Realização dos Testes](#testes)
    - [Qualidade dos Modelos](#qualidade-dos-testes-e-resultados)
- [Discussão](#discussão)

## Modelos Utilizados
- Árvore de Decisão (Decision Tree)
- Ensemble
- Floresta Randômica (Random Forest)
- Redes Neurais Convolucionais
- Regressão Linear
- Regressão Polinomial
- Support Vector Machine (SVM)

## Importando os pacotes e bibliotecas

In [2]:
# biblioteca para realizar o corte teste | treino
from sklearn.model_selection import train_test_split

# bibliotecas de classificacao
## Floresta Randomica;
## Arvore de Decisao; e
## Support Vector Machine (SVM).
from sklearn.ensemble  import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# biblioteca de modelos polinomiais
from sklearn.preprocessing import PolynomialFeatures

# biblioteca de modelos lineares
## Regressao Linear; e
## Support Vector Machine (SVM).
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

# Biblioteca auxiliar para calcular os scores
from sklearn import metrics

# bibliotecas de suporte -----
## Impressao de Graficos
from matplotlib import pyplot as plt
import seaborn as sns

## Bibliotecas Base
import pandas as pd
import numpy as np
# ----------------------------

# bibliotecas e pacotes do TensorFlow
## Redes Neurais Convolucionais
import tensorflow as tf
from keras import layers, models

2024-08-06 13:48:28.803242: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-06 13:48:28.939556: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 13:48:28.939612: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 13:48:28.975587: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-06 13:48:29.010599: I tensorflow/core/platform/cpu_feature_gua

## Importando os datasets

In [ ]:
df_residencial_raw = pd.read_csv("./databases/raw/CONSUMO MENSAL DE ENERGIA ELÉTRICA POR CLASSE - CONSUMO COMERCIAL POR UF.csv", sep = ",", index_col = 0)

## Analise Inicial dos Datasets

[Análise e limpeza dos dados dos datasets](./limpeza.ipynb)

## AED
### Análise Exploratória dos Dados

[Link para o notebook de Análise Exploratória](./analise_exploratoria.ipynb)

### Consumo Residencial por UF

## Treinando os Modelos

### Árvore de Decisão

[Link para o notebook com o modelo da Árvore de Decisão](./arvore_decisao.ipynb)

## Resultados dos Modelos

### Testes

### Qualidade dos Testes e Resultados

## Discussão

A discussão vai aqui